# Initial Set-Up

In [3]:
import pickle
import pandas as pd
import matplotlib
import os
import re

import collections
import datetime
import time

import geopandas as gpd

import numpy as np

import rasterio

from difflib import get_close_matches

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

import dask.dataframe as dd


/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Infogroup
This is where we find the weights by company location.

The code commented out below lets us scale if we do the full infogroup file.

In [4]:
df = dd.read_csv( "../../data/companyData/allIGData.csv", assume_missing=True, 
                 dtype={'parent_number': 'object','parent_employee_size_code': 'object',
                       'parent_sales_volume_code': 'object',
                       'abi': 'object'})[['archive_version_year', 'parent_number',
                       'employee_size_location','latitude', 'longitude','zipcode']]

In [4]:
df = dd.read_csv( "../../data/companyData/allIGData.csv", assume_missing=True, 
                 dtype={'parent_number': 'object','parent_employee_size_code': 'object',
                       'parent_sales_volume_code': 'object',
                       'abi': 'object'})

df.columns

FileNotFoundError: [Errno 2] No such file or directory: '/Users/brianreed/Documents/supplyChain/data/companyData/allIGData.csv'

In [5]:
df.head()

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


,archive_version_year,parent_number,employee_size_location,latitude,longitude,zipcode
0,2003.0,007508146,125.0,33.24433,-87.49823,35404.0
1,2003.0,NaN,32.0,36.06316,-90.48386,72450.0
2,2003.0,NaN,290.0,40.19310,-85.36173,47303.0
3,2003.0,441297108,168.0,33.54118,-84.55948,30213.0
4,2003.0,NaN,500.0,34.15419,-86.76856,35055.0


In [6]:
df = df[(~df.parent_number.isna()) & (~df.employee_size_location.isna())]

In [7]:
employeesByParents = pd.read_csv('../../data/companyData/totalEmployees.csv',
                                dtype={'parent_number': 'object'}).drop(columns = {'Unnamed: 0'})
employeesByParents.head()

,archive_version_year,parent_number,allEmployees
0,2003.0,000007609,0.0
1,2003.0,000023077,68.0
2,2003.0,000076547,287.0
3,2003.0,000077743,649.0
4,2003.0,000089151,188.0


In [8]:
toCalculate = df.merge(employeesByParents)

In [9]:
toCalculate.head()

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)


,archive_version_year,parent_number,employee_size_location,latitude,longitude,zipcode,allEmployees
0,2003.0,007508146,125.0,33.24433,-87.49823,35404.0,4731.0
1,2003.0,007508146,30.0,33.86970,-117.53745,92879.0,4731.0
2,2003.0,007508146,170.0,40.51728,-75.55581,18062.0,4731.0
3,2003.0,007508146,400.0,42.09046,-76.78435,14901.0,4731.0
4,2003.0,007508146,15.0,33.47081,-86.76147,35223.0,4731.0


In [10]:
toCalculate['locationFracOfEmployees'] = toCalculate['employee_size_location']/toCalculate['allEmployees']

In [11]:
fractions = toCalculate[['archive_version_year','parent_number','latitude','longitude','zipcode','locationFracOfEmployees']].\
    groupby(['archive_version_year','parent_number','latitude','longitude','zipcode']).sum().compute(workers = 100)

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=Fal

/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  return func(*args, **kwargs)
/Users/brianreed/opt/anaconda3/lib/python3.7/site-packages/dask/utils.py:29: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=Fal

In [12]:
fractions = fractions.reset_index()
fractions.head()

,archive_version_year,parent_number,latitude,longitude,zipcode,locationFracOfEmployees
0,2003.0,000023077,40.47235,-80.06152,15136.0,0.735294
1,2003.0,000076547,26.55504,-81.88423,33907.0,0.121951
2,2003.0,000077743,33.86604,-117.89996,92831.0,0.078582
3,2003.0,000077743,34.15562,-118.65163,91302.0,0.018490
4,2003.0,000089151,38.88735,-84.80038,47038.0,0.265957


In [15]:
fractions.zipcode.min()

601.0

In [13]:
fractions.to_csv('../../data/companyData/fractionEmployees_byEstablishment.csv')